In [3]:
from json import load
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key: str = os.getenv("OPENAI_API_KEY")
supabase_url: str = os.getenv("SUPABASE_URL")
supabase_key: str = os.getenv("SUPABASE_KEY")

Peter Atia Longevity Course

In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders.sitemap import SitemapLoader
import nest_asyncio
nest_asyncio.apply()

sitemap_loader = SitemapLoader(web_path="https://peterattiamd.com/sitemap.xml")

docs = sitemap_loader.load()
print(f"Loaded {len(docs)} documents")
print(docs[0].page_content[:10000])

Fetching pages: 100%|##########| 1024/1024 [01:46<00:00,  9.59it/s]


Loaded 1024 documents

 





 




Search Content - Peter Attia
































































































Skip to contentPeter AttiaNewsletter

Weekly Newsletter
The free weekly newsletter explores practical insights on longevity and health optimization. The newsletter simplifies complex research and actionable tips, making cutting-edge health knowledge accessible and relevant.

Browse



Premium Articles
These articles are produced by Peter and our dedicated team of research analysts, and cover a wide range of longevity-related topics in much greater detail than those we send in our free weekly newsletter. Peter and our research team work hard to make these articles topical, exhaustive, interesting, and, whenever possible, actionable.

Browse




Sign up for the newsletter
Sign up to receive the 5 tactics in my Longevity Toolkit, followed by non-lame weekly emails on the latest strategies and tactics for increasing your lifespan, 

Supabase

In [5]:
from supabase import create_client, Client

db: Client = create_client(supabase_url, supabase_key)